# CIS 419/519 Homework 1

Name: Jiatong Sun

Pennkey: jtsun

PennID: 53378755

In [3]:
import random 
import numpy as np
import pandas as pd
random.seed(42)  # don't change this line

In [4]:
# Load all data tables
baseDir = './'## TODO: insert path to data file
df = pd.read_csv(baseDir+'NHANES-diabetes-hw-train.csv')

# Output debugging info
print(df.shape)
df.head()


(8140, 1812)


,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,...,WHD080L,WHD110,WHD120,WHD130,WHD140,WHQ150,WHQ030M,WHQ500,WHQ520,DIABETIC
0,76195,8,2,1,18,NaN,5,7,1.0,217.0,...,NaN,NaN,NaN,NaN,138.0,18.0,NaN,NaN,NaN,0
1,76958,8,2,2,57,NaN,2,2,1.0,NaN,...,NaN,135.0,115.0,67.0,150.0,45.0,NaN,NaN,NaN,0
2,80248,8,2,2,29,NaN,2,2,2.0,NaN,...,NaN,NaN,125.0,NaN,160.0,28.0,NaN,NaN,NaN,0
3,80213,8,2,2,0,5.0,1,1,2.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,76753,8,2,1,61,NaN,3,3,2.0,NaN,...,NaN,160.0,160.0,69.0,180.0,30.0,NaN,NaN,NaN,0


In [5]:
# Print information about the dataset
print('Percentage of instances with missing features:')
print(df.isnull().sum(axis=0)/df.shape[0])
print()
print('Class information:')
print(df['DIABETIC'].value_counts())

Percentage of instances with missing features:
SEQN        0.000000
SDDSRVYR    0.000000
RIDSTATR    0.000000
RIAGENDR    0.000000
RIDAGEYR    0.000000
              ...   
WHQ150      0.407371
WHQ030M     0.853563
WHQ500      0.853563
WHQ520      0.853563
DIABETIC    0.000000
Length: 1812, dtype: float64

Class information:
0    7447
1     693
Name: DIABETIC, dtype: int64


## **Preprocessing**

The first key step in any data modeling task is cleaning your dataset. Explore your dataset and figure out what sort of preprocessing is required. Good preprocessing can make or break your final model. So choose wisely.

Some of the preprocessing steps that you can consider are :


*   One-hot encoding of variables
*   Missing value imputation
*   Removing outliers
*   Converting binary features into 0-1 representation


Feel free to reuse code you've already written in HW 0.






In [6]:
# TODO Insert your preprocessing code here

# The complete preprocessing function
def preprocessing(inputDf,missing_thresh):
    outputDf = inputDf.copy()
    # drop feature that misses too many data
    outputDf = dropMissingColumn(outputDf,missing_thresh)
    # replenish missing data
    outputDf = fillMissingColumn(outputDf)
    return outputDf

# drop the column whose missing ratio is too high
def dropMissingColumn(inputDf,missing_thresh):
    outputDf = inputDf.copy()
    missDf = getMissingRatio(outputDf)
    missIdx = missDf['MissingPercent'] >= missing_thresh
    missFeat = missDf.loc[missIdx,'Feature']
    outputDf = outputDf.drop(missFeat,axis=1)
    return outputDf

# fill the NaN
def fillMissingColumn(inputDf):
    outputDf = inputDf.copy()
    # replenish numerical with mean
    numerical_col = (outputDf.iloc[:,].dtypes!='O').values
    numericalDf = outputDf.iloc[:,numerical_col]
    numericalDf = numericalDf.fillna(numericalDf.mean())
    # replenish category with mode
    category_col = (outputDf.iloc[:,].dtypes=='O').values
    categoryDf = outputDf.iloc[:,category_col]
    categoryDf = categoryDf.fillna(categoryDf.mode().iloc[0])
    if not categoryDf.empty:
        categoryDf = pd.get_dummies(categoryDf)
    # combine two dataframes
    outputDf = pd.concat([numericalDf,categoryDf],axis=1)
    return outputDf

# calculate the missing ratio of each column
def getMissingRatio(inputDf):
    outSeries = inputDf.isna().mean()                 # calculate missing ratios
    outDf = pd.DataFrame({'Feature':outSeries.index,\
                          'MissingPercent':outSeries.values})
    return outDf

In [7]:
# Generate feature set from preprocessed data set

# Extract features w.r.t. correlation
def extractFeatures(X,y,num_feature):
    completeDf = pd.concat([X,y], axis = 1)
    corrDf = completeDf.corr().iloc[-1,:-1]
    corrDf = abs(corrDf).sort_values(ascending=False)
    feature_index = corrDf.index[:num_feature]
    X_out = X[feature_index]
    return X_out, feature_index

# Select features randomly
def randomFeatures(X,num_feature,random_seed):
    random.seed(random_seed)
    idx = np.arange(0,X.shape[1])
    random.shuffle(idx)
    X_out = X.iloc[:,idx[:10]]
    feature_index = X_out.columns
    return X_out, feature_index

In [8]:
# Preprocessing

random_seed = 42

X = df.drop(['SEQN','DIQ010','DIABETIC'],axis=1)
y = df['DIABETIC']

print(f'Diabetic Ratio (Label): {y.values.sum()/len(y):.2f}\n')

X = preprocessing(X,0.6)

Diabetic Ratio (Label): 0.09



In [9]:
# Generate three different sub data set

feat_list = list()
# feature set 1: by corr
X1, feature_1 = extractFeatures(X,y,13)
feat_list.append(feature_1)

# feature set 2: by random
X2, feature_2 = randomFeatures(X,13,random_seed)
feat_list.append(feature_2)

# feature set 3: by given example
feature_3 = ['RIDAGEYR','BMXWAIST','BMXHT','LBXTC','BMXLEG','BMXWT','BMXBMI',
             'RIDRETH1','BPQ020','ALQ120Q','DMDEDUC2','RIAGENDR','INDFMPIR']
X3 = X[feature_3]
feat_list.append(feature_3)

## **Modeling**

In this section, you are tasked with building a Decision Tree classifier to predict whether or not a patient has diabetes. The overall goal of this exercise is to investigate the dataset and develop features that would improve your model performance.

To help with this process, we have provided the structure for two helper functions. These functions will help in tuning your model as well as validating your model's performance.

Complete these two functions.



In [10]:
def cross_validated_accuracy(DecisionTreeClassifier, X, y, num_trials, num_folds, random_seed):
    random.seed(random_seed)
    """
     Args:
          DecisionTreeClassifier: An Sklearn DecisionTreeClassifier (e.g., created by "tree.DecisionTreeClassifier(criterion='entropy')")
          X: Input features
          y: Labels
          num_trials: Number of trials to run of cross validation
          num_folds: Number of folds (the "k" in "k-folds")
          random_seed: Seed for uniform execution (Do not change this) 

      Returns:
          cvScore: The mean accuracy of the cross-validation experiment

      Notes:
          1. You may NOT use the cross-validation functions provided by Sklearn
    """

    ## TODO ##
    num_correct = 0
    num_test = 0
    accuracy_list = list()
    for i in range(num_trials):
        idx = np.arange(0,X.shape[0])
        random.shuffle(idx)
        X_shuf = X.set_index(idx).sort_index()
        y_shuf = y.copy()
        y_shuf.index = idx
        y_shuf = y_shuf.sort_index()
        for j in range(num_folds):
            X_split = np.array_split(X_shuf, num_folds)
            y_split = np.array_split(y_shuf, num_folds)
            X_test = X_split.pop(j)
            y_test = y_split.pop(j)
            X_train = pd.concat(X_split)
            y_train = pd.concat(y_split)
            model = DecisionTreeClassifier.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            num_correct += (y_pred == y_test).sum()
            num_test += len(y_test)
            accuracy_list.append((y_pred == y_test).sum()/len(y_test))
    cvScore = num_correct / num_test
    x_bar = np.mean(accuracy_list)
    ssd = np.std(accuracy_list)
    num_obs = len(accuracy_list)
    t_param = 2.626   # look into the t-table
    low_bound = round(x_bar - t_param * ssd / np.sqrt(num_obs),3)
    upper_bound = round(x_bar + t_param * ssd / np.sqrt(num_obs),3)
    interval = (low_bound, upper_bound)
    print(f'Confidence Interval: {interval}')
    
    return cvScore

In [11]:
def automatic_dt_pruning(DecisionTreeClassifier, X, y, num_trials, num_folds, random_seed):
    random.seed(random_seed)
    """
    Returns the pruning parameter (i.e., ccp_alpha) with the highest cross-validated accuracy

    Args:
          DecisionTreeClassifier  : An Sklearn DecisionTreeClassifier (e.g., created by "tree.DecisionTreeClassifier(criterion='entropy')")      
          X (Pandas.DataFrame)    : Input Features
          y (Pandas.Series)       : Labels
          num_trials              : Number of trials to run of cross validation
          num_folds               : Number of folds for cross validation (The "k" in "k-folds") 
          random_seed             : Seed for uniform execution (Do not change this)


    Returns:
          ccp_alpha : Tuned pruning paramter with highest cross-validated accuracy

    Notes:
        1. Don't change any other Decision Tree Classifier parameters other than ccp_alpha
        2. Use the cross_validated_accuracy function you implemented to find the cross-validated accuracy

    """


    ## TODO ##
    ccp_idx = np.linspace(0,1,101)
    last_accuracy = 0
    for cur_ccp in ccp_idx:
        DecisionTreeClassifier.ccp_alpha = cur_ccp
        cur_accuracy = cross_validated_accuracy(DecisionTreeClassifier,
                            X, y, num_trials, num_folds, random_seed)
        print(f'ccp_alpha = {cur_ccp:.2f}: {round(cur_accuracy,3)}\n')
        if last_accuracy - cur_accuracy > 0.01:
            return (cur_ccp - 0.01)
        else:
            last_accuracy = cur_accuracy
    ccp_alpha = cur_ccp
    
    return ccp_alpha

## **Tuning and Testing**

With the helper functions and your processed dataset, build a Decision Tree classifier to classify Diabetic patients and tune it to maximize model performance.

Once you are done with your modeling process, test your model on the test dataset and output your predictions in a file titled "cis519_hw1_predictions.csv", with one row per prediction.

In [12]:
from sklearn import tree

In [13]:
# Set learning parameters
num_trials = 10
num_folds = 10
T = tree.DecisionTreeClassifier(criterion='entropy')

In [14]:
## TODO ##

# Cross-Validation on three data set
# Feature set 1: by corr
cvScore_1 = cross_validated_accuracy(T, 
               X1, y, num_trials, num_folds, random_seed)
print(f'feature 1 accuracy: {cvScore_1:.3f}')

# Feature set 2: by random
cvScore_2 = cross_validated_accuracy(T, 
               X2, y, num_trials, num_folds, random_seed)
print(f'feature 2 accuracy: {cvScore_2:.3f}')

# Feature set 3: by given example
cvScore_3 = cross_validated_accuracy(T, 
               X3, y, num_trials, num_folds, random_seed)
print(f'feature 3 accuracy: {cvScore_3:.3f}\n')

# Compare the accuracies and choose the best feature set
best_index = np.argmax([cvScore_1,cvScore_2,cvScore_3])
best_accuracy = max([cvScore_1,cvScore_2,cvScore_3])
best_feat = feat_list[best_index]

Confidence Interval: (0.927, 0.931)
feature 1 accuracy: 0.929
Confidence Interval: (0.862, 0.868)
feature 2 accuracy: 0.865
Confidence Interval: (0.875, 0.881)
feature 3 accuracy: 0.878



In [15]:
# Prune the tree
X_train = X[best_feat]
best_ccp = automatic_dt_pruning(T, 
                X_train, y, num_trials, num_folds, random_seed)
# Show result
print(f'\nbest feature set: feature_{best_index+1}')
print(f'best accuracy: {round(best_accuracy,3)}')
print(f'best feature:\n {best_feat.values}\n')
print(f'best ccp_alpha: {round(best_ccp,2)}\n')

Confidence Interval: (0.926, 0.931)
ccp_alpha = 0.00: 0.929

Confidence Interval: (0.94, 0.944)
ccp_alpha = 0.01: 0.942

Confidence Interval: (0.934, 0.938)
ccp_alpha = 0.02: 0.936

Confidence Interval: (0.934, 0.938)
ccp_alpha = 0.03: 0.936

Confidence Interval: (0.934, 0.938)
ccp_alpha = 0.04: 0.936

Confidence Interval: (0.934, 0.938)
ccp_alpha = 0.05: 0.936

Confidence Interval: (0.934, 0.938)
ccp_alpha = 0.06: 0.936

Confidence Interval: (0.934, 0.938)
ccp_alpha = 0.07: 0.936

Confidence Interval: (0.934, 0.938)
ccp_alpha = 0.08: 0.936

Confidence Interval: (0.934, 0.938)
ccp_alpha = 0.09: 0.936

Confidence Interval: (0.934, 0.938)
ccp_alpha = 0.10: 0.936

Confidence Interval: (0.934, 0.938)
ccp_alpha = 0.11: 0.936

Confidence Interval: (0.934, 0.938)
ccp_alpha = 0.12: 0.936

Confidence Interval: (0.934, 0.938)
ccp_alpha = 0.13: 0.936

Confidence Interval: (0.933, 0.938)
ccp_alpha = 0.14: 0.935

Confidence Interval: (0.913, 0.919)
ccp_alpha = 0.15: 0.916


best feature set: featur

In [16]:
# Making predictions on unlabelled data set
X_unlabel = pd.read_csv('hw1-NHANES-diabetes-test-unlabeled.csv')
X_unlabel = preprocessing(X_unlabel,1)
X_unlabel = X_unlabel[best_feat]
T.ccp_alpha = best_ccp
T = T.fit(X_train, y)
y_pred = T.predict(X_unlabel)
predDf = pd.Series(y_pred).rename('DIABETIC')
predDf.to_csv('cis519_hw1_predictions.csv',header = False,index = False)
print(f'Diabetic Ratio (Unabel): {y_pred.sum()/len(y_pred):.2f}\n')

Diabetic Ratio (Unabel): 0.07

